## 0. Install and Import Modules

Ensure `torch` and `torchvision` are installed. If `pytorch3d` is not installed, install it using the following cell:

In [ ]:
# import os
# import sys
# import torch
# need_pytorch3d=False
# try:
#     import pytorch3d
# except ModuleNotFoundError:
#     need_pytorch3d=True
# if need_pytorch3d:
#     if torch.__version__.startswith("1.13.") and sys.platform.startswith("linux"):
#         # We try to install PyTorch3D via a released wheel.
#         pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
#         version_str="".join([
#             f"py3{sys.version_info.minor}_cu",
#             torch.version.cuda.replace(".",""),
#             f"_pyt{pyt_version_str}"
#         ])
#         !pip install fvcore iopath
#         !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
#     else:
#         # We try to install PyTorch3D from source.
#         !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
#         !tar xzf 1.10.0.tar.gz
#         os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
#         !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 KB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61431 sha256=2a5a735959b1ed1a5c444b3f0a8b8c4fc0d7f2c5ffc6c6c47b9fce8c6fd6784d
  Stored in directory: /root/.cache/pip/wheels/b8/79/07/c0e9367f5b5ea325e246bd73651e8af175fabbef943043b1cc
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31547 sha256=201007029374099283742884f8d86469d1265b3240302b52f894036335fd8d5b
  Stored in directory: /root/.cache/pip/wheels/89/3e/24/0f349c0b2eeb6965903035f3b00dbb5c9bea437b4a2f18d82c
Successfully built fvcore iopath
Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py38_cu116_pyt1131/dow

In [ ]:
# # imports
# import numpy as np
# import torch
# from pytorch3d.transforms.so3 import (
#     so3_exp_map,
#     so3_relative_angle,
# )
# from pytorch3d.transforms.se3 import (
#     se3_exp_map,
#     se3_log_map,
# )

# import pytorch3d.transforms as transforms
    
# # add path for demo utils
# import sys
# import os
# sys.path.append(os.path.abspath(''))

# # set for reproducibility
# torch.manual_seed(42)
# if torch.cuda.is_available():
#     device = torch.device("cuda:0")
# else:
#     device = torch.device("cpu")
#     print("WARNING: CPU only, this will be slow!")

## **Data preparation**

In [ ]:
# # Load the matrices 
# T_o_cb_batch = np.load('')
# batch_size = T_o_cb_batch.shape[0]
# T_o_p_batch = np.load('')
# T_c_p_batch = np.load('')
# X_matrix = np.load('')
# X_batch = np.tile(X_matrix, [batch_size,1,1])

# # Load into torch tensor
# X_batch = torch.tensor(X_batch)
# T_o_p_batch = torch.tensor(T_o_p_batch)
# T_o_cb_batch = torch.tensor(T_o_cb_batch)
# T_c_p_batch = torch.tensor(T_c_p_batch)

In [ ]:
# ## test
# from scipy.spatial.transform import Rotation as R

# batch_size = 10
# T_o_cb_batch = np.tile(np.eye(4), [batch_size,1,1])
# T_o_p_batch = np.tile(np.eye(4), [batch_size,1,1])
# T_c_p_batch = np.tile(np.eye(4), [batch_size,1,1])
# X_matrix = np.hstack((np.vstack((R.random().as_matrix(), np.zeros([1,3]))), np.ones([4,1])))
# X_batch = np.tile(X_matrix, [batch_size,1,1])
# X_batch[:, :3, 3] = 2
# print(X_batch[1, :,:])
# X_batch = torch.tensor(X_batch)
# T_o_p_batch = torch.tensor(T_o_p_batch)
# T_o_cb_batch = torch.tensor(T_o_cb_batch)
# T_c_p_batch = torch.tensor(T_c_p_batch)

[[-0.49508795 -0.8619929  -0.10888599  2.        ]
 [-0.86414445  0.47551297  0.16474763  2.        ]
 [-0.09023459  0.17565779 -0.98030712  2.        ]
 [ 0.          0.          0.          1.        ]]


In [ ]:
# # Create Transform3d objects from the batch of transformation matrices
# T_o_cb = transforms.Transform3d(matrix=T_o_cb_batch, device=device)
# X = transforms.Transform3d(matrix=X_batch, device=device)
# T_o_p = transforms.Transform3d(matrix=T_o_p_batch, device=device)
# T_c_p = transforms.Transform3d(matrix=T_c_p_batch, device=device)

# # cam_gt: the ground truth camera pose w.r.t the optical tracker cam_gt = T_o_p * (T_c_p)^-1
# cam_gt = T_o_cb.compose(X)

# # cam_real: the camera pose from initial hand-eye calibration cam_real = T_o_cb * X
# cam_real = T_o_p.compose(T_c_p.inverse())

## **Define optimization loss function**



In [ ]:
# def camera_distance(hand_eye_real, hand_eye_gt):
#   '''
#   Calculate the l2 distance in se(3) manifold for the camera.
#   distance = sum||log(hand_eye_real*hand_eye_gt)||_2
#   '''

#   # Convert a batch of 4x4 transformation matrices transform to 
#   # a batch of 6-dimensional SE(3) logarithms of the SE(3) matrices
#   # print(hand_eye_real.compose(hand_eye_gt.inverse()).get_matrix())
#   tmp_matrix = hand_eye_real.compose(hand_eye_gt.inverse()).get_matrix()
#   vec = se3_log_map(tmp_matrix.permute(0,2,1)).sum(0)
#   # print(torch.norm(vec))

#   return torch.norm(vec)
  

# # camera_distance(cam_real, cam_gt)

## **Optimization**

In [ ]:
# # Create Transform3d objects from the batch of transformation matrices
# T_o_cb = transforms.Transform3d(matrix=T_o_cb_batch, device=device)
# X = transforms.Transform3d(matrix=X_batch, device=device)
# T_o_p = transforms.Transform3d(matrix=T_o_p_batch, device=device)
# T_c_p = transforms.Transform3d(matrix=T_c_p_batch, device=device)

# # hand_eye_real: the hand-eye transformation from initial calibration X
# hand_eye_real_6D = se3_log_map(X.get_matrix().permute(0, 2, 1))
# hand_eye_real_6D.requires_grad = True


# # hand_eye_gt: the hand-eye transformation from the chain 
# # hand_eye_gt = (T_o_cb)^-1 * T_o_p * (T_c_p)^-1
# hand_eye_gt = T_o_cb.inverse().compose(T_o_p).compose(T_c_p.inverse())

# # init the optimizer
# optimizer = torch.optim.SGD([hand_eye_real_6D], lr=.001, momentum=0.9)

# # run the optimization
# n_iter = 2000  # fix the number of iterations
# for it in range(n_iter):
#     # re-init the optimizer gradients
#     optimizer.zero_grad()

#     # transform the 6d vector into 4x4 matrix
#     hand_eye_real = se3_exp_map(hand_eye_real_6D).permute(0, 2, 1)
#     hand_eye_real = transforms.Transform3d(matrix=hand_eye_real, device=device)

#     # compare the composed cameras with the ground truth relative cameras
#     # camera_distance corresponds to $d$ from the description
#     loss = \
#         camera_distance(hand_eye_real, hand_eye_gt)
#     # loss.requires_grad_(True)

#     # our loss function is the camera_distance
#     loss.backward()
    
#     # apply the gradients
#     optimizer.step()

#     # # plot and print status message
#     if it % 200==0 or it==n_iter-1:
#         status = 'iteration=%3d; camera_distance=%1.3e' % (it, loss)
#         print(status)
        
# print('Optimization finished.')

iteration=  0; camera_distance=4.696e+01
iteration=200; camera_distance=2.786e+01
iteration=400; camera_distance=7.859e+00
iteration=600; camera_distance=5.260e-04
iteration=800; camera_distance=1.630e-02
iteration=1000; camera_distance=7.543e-03
iteration=1200; camera_distance=1.241e-02
iteration=1400; camera_distance=7.818e-03
iteration=1600; camera_distance=1.311e-02
iteration=1800; camera_distance=3.938e-03
iteration=1999; camera_distance=4.000e-03
Optimization finished.


## **Pypose implementation**

In [ ]:
# !pip install pypose

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 KB 10.9 MB/s eta 0:00:00


In [1]:
import os
import sys
import numpy as np
import torch, pypose as pp
from torch import nn
from scipy.spatial.transform import Rotation as R

/home/shc/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

## **Data Preparation**

In [ ]:
# # Load the matrices 
# T_o_cb_batch = np.load('')
# batch_size = T_o_cb_batch.shape[0]
# T_o_p_batch = np.load('')
# T_c_p_batch = np.load('')
# X_matrix = np.load('')
# X_batch = np.tile(X_matrix, [batch_size,1,1])

# convert to pp tensor
# X = pp.mat2SE3(X_batch)
# T_o_p = pp.mat2SE3(T_o_p_batch)
# T_o_cb = pp.mat2SE3(T_o_cb_batch)
# T_c_p = pp.mat2SE3(T_c_p_batch)

In [3]:
# test
batch_size = 800
T_o_cb_batch = np.tile(np.eye(4), [batch_size,1,1])
T_o_p_batch = np.tile(np.eye(4), [batch_size,1,1])
T_c_p_batch = np.tile(np.eye(4), [batch_size,1,1])
X_matrix = np.hstack((np.vstack((R.random().as_matrix(), np.zeros([1,3]))), np.ones([4,1])))
X_batch = np.tile(X_matrix, [batch_size,1,1])
X_batch[:, :3, 3] = 2.5

print(X_batch[1, :,:])

# convert to pp tensor
X = pp.mat2SE3(X_batch)
T_o_p = pp.mat2SE3(T_o_p_batch)
T_o_cb = pp.mat2SE3(T_o_cb_batch)
T_c_p = pp.mat2SE3(T_c_p_batch)
# print(X_batch[1,...])
# print(X_batch @ pp.Inv(X_batch))

[[ 0.61301403 -0.2401426   0.75269205  2.5       ]
 [ 0.7726237  -0.01690138 -0.63463924  2.5       ]
 [ 0.16512545  0.97059048  0.17517908  2.5       ]
 [ 0.          0.          0.          1.        ]]


## **Optimization**

In [4]:
class CamDis(nn.Module):
    def __init__(self, hand_eye_real_6D):
        super().__init__()
        self.real = pp.Parameter(hand_eye_real_6D)

    def forward(self, input):
        # input hand_eye_gt
        hand_eye_real = self.real.Exp()
        tmp_quats = hand_eye_real @ pp.Inv(input)
        vec = tmp_quats.Log()
        # print(hand_eye_gt[0])
        t = tmp_quats[:, :3].mean(0)
        rot_vec = vec[:, 3:].mean(0)
        return torch.norm(rot_vec) + torch.norm(t)

In [5]:
# hand_eye_real: the hand-eye transformation from initial calibration X
hand_eye_real = X
hand_eye_real_6D = hand_eye_real.Log().to(device)

# hand_eye_gt: the hand-eye transformation from the chain 
# hand_eye_gt = (T_o_cb)^-1 * T_o_p * (T_c_p)^-1
hand_eye_gt = pp.Inv(T_o_cb) @ T_o_p @ pp.Inv(T_c_p)

# init the optimizer
camera_distance = CamDis(hand_eye_real_6D).to(device)
input = hand_eye_gt.to(device)
strategy = pp.optim.strategy.Adaptive(damping=1e-6)
optimizer = pp.optim.LM(camera_distance, strategy=strategy)

# run the optimization
n_iter = 200  # fix the number of iterations
for it in range(n_iter):
    loss = optimizer.step(input)

    # # plot and print status message
    if it % 5==0 or it==n_iter-1:
        status = 'iteration=%3d; camera_distance=%1.3e' % (it, loss)
        print(status)
    if loss < 1e-6:
        print('Early Stopping with loss:', loss.item())
        break
print(pp.matrix(hand_eye_real_6D[0].Exp()), input[0])
print('Optimization finished.')


/home/shc/anaconda3/envs/pytorch3d/lib/python3.9/site-packages/pypose/lietensor/lietensor.py:918: UserWarning: Tensor Shape Invalid by calling <slot wrapper '__getitem__' of 'torch._C._TensorBase' objects>, go to https://pypose.org/docs/main/generated/pypose.LieTensor
  warnings.warn('Tensor Shape Invalid by calling {}, ' \


iteration=  0; camera_distance=1.467e+01
iteration=  5; camera_distance=7.036e-02
iteration= 10; camera_distance=6.477e-03
iteration= 15; camera_distance=8.748e-04
iteration= 20; camera_distance=1.183e-04
iteration= 25; camera_distance=1.569e-05
iteration= 30; camera_distance=9.219e-07
Early Stopping with loss: 9.219348534101693e-07
tensor([[ 1.0000e+00, -3.5636e-04, -1.2756e-04,  2.5622e-04],
        [ 3.5640e-04,  1.0000e+00,  3.4686e-04,  2.6208e-04],
        [ 1.2743e-04, -3.4690e-04,  1.0000e+00,  2.5407e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]], device='cuda:0',
       dtype=torch.float64) SE3Type LieTensor:
LieTensor([0., 0., 0., 0., 0., 0., 1.], device='cuda:0', dtype=torch.float64)
Optimization finished.
